In [87]:
import os
import cv2
import pickle
import random as rnd
import numpy as np
import pandas as pd

In [88]:
# Image Preprocessing Function with default image size
def preprocess(image_path, image_size=224):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (image_size, image_size))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.0
    return torch.tensor(image).permute(2, 0, 1)

In [89]:
# Create DataFrame from Directory Structure
def create_dataframe(datadir):
    result_dataframe = []
    for train_test_name in os.listdir(datadir):
        label_dir = os.path.join(datadir, train_test_name)
        for label_name in os.listdir(label_dir):
            image_dir = os.path.join(label_dir, label_name)
            for image_path in os.listdir(image_dir):
                record_dict = {
                    'is_train': 0 if train_test_name == 'test' else 1,
                    'label': label_name.replace('healthy', 'Healthy'),
                    'image_path': os.path.join(image_dir, image_path)
                }
                result_dataframe.append(record_dict)
    return result_dataframe


In [90]:
# Data Loading and Preparation
datadir = '/kaggle/input/lungs-disease-data/data'
result_dataframe = create_dataframe(datadir)
result_dataframe = pd.DataFrame.from_records(result_dataframe)  # Ensure this conversion is correctly done
train_dataframe = result_dataframe[result_dataframe['is_train'] == 1].drop('is_train', axis=1)
test_dataframe = result_dataframe[result_dataframe['is_train'] == 0].drop('is_train', axis=1)

In [91]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_dataframe['label'] = le.fit_transform(train_dataframe['label'].values)
test_dataframe['label'] = le.transform(test_dataframe['label'].values)

In [92]:
train_dataframe['label'].value_counts()

label
1    111
0     70
2     70
Name: count, dtype: int64

In [93]:
test_dataframe['label'].value_counts()

label
1    26
0    20
2    20
Name: count, dtype: int64

In [94]:
test_dataframe['label'].value_counts().keys() == train_dataframe['label'].value_counts().keys()

array([ True,  True,  True])

In [95]:
# Data Splitting
from sklearn.model_selection import train_test_split

train_images, validation_images, train_labels, validation_labels = train_test_split(train_dataframe['image_path'].values, train_dataframe['label'].values, test_size=0.2, stratify=train_dataframe['label'].values, random_state=42, shuffle=True)
test_images, test_labels = test_dataframe['image_path'].values, test_dataframe['label'].values


In [96]:
print(train_images.shape, train_labels.shape)
print(validation_images.shape, validation_labels.shape)
print(test_images.shape, test_labels.shape)

(200,) (200,)
(51,) (51,)
(66,) (66,)


In [97]:
print(np.unique(train_labels,return_counts=True))
print(np.unique(validation_labels,return_counts=True))
print(np.unique(test_labels,return_counts=True))

(array([0, 1, 2]), array([56, 88, 56]))
(array([0, 1, 2]), array([14, 23, 14]))
(array([0, 1, 2]), array([20, 26, 20]))


In [98]:
import torch
from torch import nn, optim
import torchvision
from torchvision import models, transforms, datasets
from torchvision.models import VGG16_Weights
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

In [99]:
test_images, test_labels = test_dataframe['image_path'].values, test_dataframe['label'].values

In [100]:
# Define Transformations for Data Augmentation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [101]:
# Model Definition with Corrected Pretrained Argument
vgg16_model = models.vgg16(pretrained=True)
for param in vgg16_model.features.parameters():
    param.requires_grad = False
num_features = vgg16_model.classifier[6].in_features
vgg16_model.classifier[6] = nn.Linear(num_features, len(np.unique(train_labels)))

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [102]:
# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16_model.classifier.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [103]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
vgg16_model = vgg16_model.to(device)

Using device: cuda


In [104]:
def calculate_metrics(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    return f1, precision, recall

In [105]:
# Training and Evaluation Functions
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        true_labels, predictions = [], []
        for image_paths, labels in DataLoader(list(zip(train_images, train_labels)), batch_size=32, shuffle=True):
            inputs = torch.stack([transform(preprocess(path)) for path in image_paths]).to(device)
            labels = torch.from_numpy(np.array(labels)).to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            true_labels.extend(labels.tolist())
            predictions.extend(preds.tolist())

        scheduler.step()
        f1, precision, recall = calculate_metrics(true_labels, predictions)
        print(f'Epoch {epoch+1}/{num_epochs}: F1={f1:.2f}, Precision={precision:.2f}, Recall={recall:.2f}')

In [106]:
def evaluate_model(model, image_paths, labels):
    model.eval()
    true_labels, predictions = [], []
    inputs = torch.stack([transform(preprocess(path)) for path in image_paths]).to(device)
    labels = torch.from_numpy(np.array(labels)).to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    true_labels.extend(labels.tolist())
    predictions.extend(predicted.tolist())

    f1, precision, recall = calculate_metrics(true_labels, predictions)
    cr = classification_report(true_labels, predictions)
    print(f'Accuracy: {100 * sum(np.array(true_labels) == np.array(predictions)) / len(labels):.2f}%')
    print(f'Classification Report:\n{cr}')

In [107]:
if __name__ == '__main__':
    train_model(vgg16_model, criterion, optimizer, scheduler, num_epochs=25)
    evaluate_model(vgg16_model, test_images, test_labels)

Epoch 1/25: F1=0.58, Precision=0.59, Recall=0.58
Epoch 2/25: F1=0.76, Precision=0.77, Recall=0.76
Epoch 3/25: F1=0.66, Precision=0.67, Recall=0.67
Epoch 4/25: F1=0.71, Precision=0.72, Recall=0.71
Epoch 5/25: F1=0.76, Precision=0.76, Recall=0.76
Epoch 6/25: F1=0.70, Precision=0.70, Recall=0.70
Epoch 7/25: F1=0.61, Precision=0.61, Recall=0.61
Epoch 8/25: F1=0.76, Precision=0.77, Recall=0.76
Epoch 9/25: F1=0.74, Precision=0.75, Recall=0.73
Epoch 10/25: F1=0.77, Precision=0.78, Recall=0.76
Epoch 11/25: F1=0.77, Precision=0.77, Recall=0.77
Epoch 12/25: F1=0.77, Precision=0.76, Recall=0.77
Epoch 13/25: F1=0.71, Precision=0.71, Recall=0.71
Epoch 14/25: F1=0.79, Precision=0.82, Recall=0.78
Epoch 15/25: F1=0.79, Precision=0.82, Recall=0.80
Epoch 16/25: F1=0.84, Precision=0.84, Recall=0.85
Epoch 17/25: F1=0.82, Precision=0.83, Recall=0.82
Epoch 18/25: F1=0.89, Precision=0.89, Recall=0.89
Epoch 19/25: F1=0.85, Precision=0.85, Recall=0.85
Epoch 20/25: F1=0.84, Precision=0.84, Recall=0.83
Epoch 21/